# Evaluate Problematic issues

In [ ]:
import geopandas as gpd
import folium
import json
from tqdm import tqdm
import re
import plotly.express as px
import ruptures as rpt 
from sklearn.metrics import r2_score

from custom_functions.gtfs_methods import detect_direction, match_schedule_for_service_line
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

In [ ]:
res = pd.read_pickle("data/computed/matches_clean_EWT_delay.pkl")

In [ ]:
sample_7_unbiased = res[(res.route_short_name == "7") & (res.date_label == "workdays") & (res.direction_id == 0)]
test = sample_7_unbiased[(sample_7_unbiased.trip_id == 113338966237534000) & (sample_7_unbiased.date_normalized == "14-09-2021")].sort_values("stop_sequence")
test

#test = sample_7[(sample_7.trip_id == 113338889237534000) & (sample_7.date_normalized == "08-09-2021")].sort_values("stop_sequence")
fig = px.scatter(sample_7_unbiased, x="stop_sequence", y="delay", color = "date_normalized", symbol="direction_id", hover_data=['stop_id', "stop_name", "direction_id", "stop_sequence"])
fig.show()


In [ ]:
test_day = sample_7_unbiased[((sample_7_unbiased.date_normalized == "14-09-2021"))][["trip_id", "delay", "stop_sequence"]].groupby(["stop_sequence"], as_index = False).median()
fig = px.scatter(test_day, x="stop_sequence", y="delay")
fig.show()

In [ ]:
order = 15

p = np.polyfit(test_day.sort_values(by="stop_sequence")["stop_sequence"],test_day.sort_values(by="stop_sequence")["delay"], order)
y = np.polyval(p,test.sort_values(by="stop_sequence")["stop_sequence"])
fig = px.scatter(x=test.sort_values(by="stop_sequence")["stop_sequence"], y =y)
fig.show()

In [ ]:
from sklearn.metrics import r2_score


for order in range(1, 20):
    p = np.polyfit(test_day.sort_values(by="stop_sequence")["stop_sequence"],test_day.sort_values(by="stop_sequence")["delay"], order)
    y = np.polyval(p,test_day.sort_values(by="stop_sequence")["stop_sequence"])
    print(order , " : ", r2_score(test_day.sort_values(by="stop_sequence")["delay"], y))

### Test

In [ ]:
from sklearn.metrics import r2_score


for order in range(1, 15):
    p = np.polyfit(test.sort_values(by="stop_sequence")["stop_sequence"],test.sort_values(by="stop_sequence")["delay"], order)
    y = np.polyval(p,test.sort_values(by="stop_sequence")["stop_sequence"])
    print(order , " : ", r2_score(test.sort_values(by="stop_sequence")["delay"], y))

In [ ]:
order = 9

p = np.polyfit(test.sort_values(by="stop_sequence")["stop_sequence"],test.sort_values(by="stop_sequence")["delay"], order)
y = np.polyval(p,test.sort_values(by="stop_sequence")["stop_sequence"])

order2 = 7

p2 =  np.polyfit(test.sort_values(by="stop_sequence")["stop_sequence"], y, order2)
y2 = np.polyval(p2,test.sort_values(by="stop_sequence")["stop_sequence"])
fig = px.scatter(x=test.sort_values(by="stop_sequence")["stop_sequence"], y =y2)
fig.show()

In [ ]:
y, y2

In [ ]:

for order in range(1, 15):
    p2 = np.polyfit(test.sort_values(by="stop_sequence")["stop_sequence"],y, order)
    y2 = np.polyval(p2,test.sort_values(by="stop_sequence")["stop_sequence"])
    print(order , " : ", r2_score(y, y2))

## Full method 

In [ ]:
import geopandas as gpd

In [ ]:
lines = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
stops_geom = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")

In [ ]:
stops_geom

In [ ]:
test = stops_geom[["Code_Ligne", "Variante", "stop_id"]].groupby(["Code_Ligne", "Variante", "stop_id"], as_index=False).count()
test
test.groupby(["Code_Ligne", "stop_id"], as_index = False).count().sort_values("Variante", ascending = False )

In [ ]:
def detect_direction(
    ligne: str, geom: pd.DataFrame, schedule: pd.DataFrame
) -> pd.DataFrame:
    g = geom[(geom.ligne_cleaned == ligne)]
    g_dir = pd.DataFrame()

    for name, group in g.groupby("VARIANTE"):
        sched_stops = schedule.sort_values("time_seconds").stop_name.to_list()

        hset_fr = group.sort_values("succession").stop_id.to_list()
        hset_nl = group.sort_values("succession").descr_nl.to_list()

        for i in range(len(hset_fr) - len(sched_stops) + 1):
            if (sched_stops == hset_fr[i : i + len(sched_stops)]) | (
                sched_stops == hset_nl[i : i + len(sched_stops)]
            ):
                g_dir = group

    return g_dir

In [ ]:
stops_geom[stops_geom["numero_lig"] == 1]

In [ ]:
stg = stops_geom.groupby(["numero_lig"])

In [ ]:
res_light = res[["date_normalized", "route_short_name", "stop_id", "delay", "stop_sequence", "direction_id"]].groupby(["route_short_name", "direction_id"])

In [ ]:
res_g_test = res.groupby(["trip_id", "date_normalized"])

In [ ]:
count = 0
index = 0
br = 0
rec = []
for name, sched in tqdm(res_g_test):
    if True:
        if sched.shape[0]>1:
            sched_stops = sched.sort_values("stop_sequence").stop_name.to_list()[:2]
            geo = stg.get_group(int(sched.iloc[0].route_short_name))
            var = pd.DataFrame()

            for name, group in geo.groupby(["Variante"]):

                hset_fr = group.sort_values("succession").descr_fr.to_list()
                hset_nl = group.sort_values("succession").descr_nl.to_list()
                try:
                    try:
                        i1 = hset_fr.index(sched_stops[0])
                    except:
                        i1 = hset_nl.index(sched_stops[0])

                    try:
                        i2 = hset_fr.index(sched_stops[1])
                    except:
                        i2 = hset_nl.index(sched_stops[1])
                except:
                    #print(sched.shape)
                    pass
                    
                if i1<i2:
                    var = group
                    index += 1

            if var.empty :
                #print(sched_stops, hset_fr, hset_nl)
                pass
            else:
                rec.extend(sched.merge(
                    var[["succession", "stop_id"]],
                    "left",
                    "stop_id"
                ).to_dict("record"))

    #br += 1
    count += 1

count, index

## Clean Successions

In [ ]:
seq_data = pd.DataFrame(rec)
seq_data.to_pickle("data/computed/matches_clean_EWT_delay_seq.pkl")

In [ ]:
seq = pd.read_pickle("data/computed/matches_clean_EWT_delay_seq.pkl")

In [ ]:
seq_f = seq[seq.regularity == 1][["route_short_name", "stop_name" ,"date_normalized", "direction_id", "succession", "EWT"]].groupby(["route_short_name", "stop_name", "date_normalized", "direction_id", "succession"], as_index = False).median()

#### Sample 1

In [ ]:
route = "7"
direction = 1

sample_date = seq_f[(seq_f.route_short_name == route) & (seq_f.direction_id == direction)].sort_values("succession")
fig = px.scatter(sample_date, x="succession", y="EWT", hover_data=["stop_name"], color = "date_normalized")
fig.show()

In [ ]:
route = "7"
date = "08-09-2021"
direction = 1

sample = seq_f[(seq_f.route_short_name == route) & (seq_f.date_normalized == date) & (seq_f.direction_id == direction)].sort_values("succession")
fig = px.scatter(sample, x="succession", y="EWT", hover_data=["stop_name"])
fig.show()

In [ ]:
thresh = 0.90
for order in range(1, 50):
    p = np.polyfit(sample.sort_values(by="succession")["succession"],sample.sort_values(by="succession")["EWT"], order)
    y = np.polyval(p,sample.sort_values(by="succession")["succession"])
    t = r2_score(sample.sort_values(by="succession")["EWT"], y)
    if t > thresh :
        print(order , " : ", t)
        order_f = order
        break

p = np.polyfit(sample.sort_values(by="succession")["succession"],sample.sort_values(by="succession")["EWT"], order_f)
y = np.polyval(p,sample.sort_values(by="succession")["succession"])
fig = px.scatter(x=sample.sort_values(by="succession")["succession"], y=y)
fig.show()


In [ ]:
sample["smoothen_delay"] = y
sample["growth"] = sample["smoothen_delay"] - sample["smoothen_delay"].shift(1)

thresh_growth = 0.2
cut_off = np.std(sample["growth"])*thresh_growth

sample["growth_cut"] = np.where(
    sample["growth"].abs() >= cut_off,
    sample["growth"],
    np.NaN
)

sample["lab_growth"] = np.where(
    np.isnan(sample["growth_cut"]) == True,
    np.NaN,
    np.where(
         (sample["growth_cut"] > 0),
         1,
         -1
    )
)

sample.drop(columns=["smoothen_delay", "growth"])

## All Samples

In [ ]:
seq = pd.read_pickle("data/computed/matches_clean_EWT_delay_seq.pkl")
seq_f = seq[["route_short_name", "stop_name" ,"date_normalized", "trip_headsign", "succession", "EWT"]].groupby(["route_short_name", "stop_name", "date_normalized", "trip_headsign", "succession"], as_index = False).median()

In [ ]:
seq_g = seq_f.groupby(["route_short_name", "date_normalized", "trip_headsign"])

In [ ]:
res = []
for name, group in tqdm(seq_g):
    group = group.sort_values("succession")
    group = group.dropna()
    try :
        if group.empty == False:
            thresh = 0.95
            for order in range(1, 50):
                
                p = np.polyfit(group.sort_values(by="succession")["succession"],group.sort_values(by="succession")["EWT"], order)
                y = np.polyval(p,group.sort_values(by="succession")["succession"])
                t = r2_score(group.sort_values(by="succession")["EWT"], y)
                if t > thresh :
                    order_f = order
                    break

            p = np.polyfit(group.sort_values(by="succession")["succession"],group.sort_values(by="succession")["EWT"], order_f)

            group["smoothen_delay"] = np.polyval(p,group.sort_values(by="succession")["succession"])
            group["growth"] = group["smoothen_delay"] - group["smoothen_delay"].shift(1)

            thresh_growth = 0.10
            cut_off = np.std(group["growth"])*thresh_growth

            group["growth_cut"] = np.where(
                group["growth"].abs() >= cut_off,
                group["growth"],
                np.NaN
            )

            group["lab_growth"] = np.where(
                np.isnan(group["growth_cut"]) == True,
                np.NaN,
                np.where(
                    (group["growth_cut"] > 0),
                    1,
                    -1
                )
            )

            group.drop(columns=["smoothen_delay", "growth"])
            res.extend(group.to_dict("records"))
    except Exception as e :
        print(name)
        print(group)
        print(e)
        break

    

In [ ]:
res_d = pd.DataFrame(res)

In [ ]:
res_d.to_pickle("data/computed/lines_growth_EWT_th.pkl")

In [ ]:
res_g = res_d.groupby(["route_short_name", "date_normalized", "direction_id"])

In [ ]:
test.sort_values("succession")

In [ ]:
test = res_g.get_group(("1", "08-09-2021", 0))
test["lab_growth"] = test["lab_growth"].astype("str")
fig = px.scatter(test, x="succession", y="EWT", hover_data=["stop_name"], color="lab_growth", color_continuous_scale="portland")
fig.show()

In [ ]:
thresh = 0.95
for order in range(1, 50):
    p = np.polyfit(test.sort_values(by="succession")["succession"],test.sort_values(by="succession")["delay"], order)
    y = np.polyval(p,test.sort_values(by="succession")["succession"])
    t = r2_score(test.sort_values(by="succession")["delay"], y)
    if t > thresh :
        print(order , " : ", t)
        order_f = order
        break

p = np.polyfit(test.sort_values(by="succession")["succession"],test.sort_values(by="succession")["delay"], order_f)
y = np.polyval(p,test.sort_values(by="succession")["succession"])
fig = px.scatter(x=test.sort_values(by="succession")["succession"], y=y)
fig.show()
